In [1]:
def draw_bipartite(B, **kwargs):
    # Separate by group
    #l, r = nx.bipartite.sets(B)
    l = [n for n,d in B.nodes(data=True) if d.get('bipartite') == 0]
    r = [n for n,d in B.nodes(data=True) if d.get('bipartite') == 1]
    pos = {}

    # Update position for node from each group
    pos.update((node, (1, index)) for index, node in enumerate(l))
    pos.update((node, (2, index)) for index, node in enumerate(r))

    nx.draw(B, pos=pos, **kwargs)

In [2]:
# Самая простая реализация с перебором. На >10 вершинах работает бесконечно долго.
def solve_tsp(G):
    paths = itertools.permutations(list(G.nodes), G.number_of_nodes())
    
    min_weight = math.inf
    min_path = None
    count = 0
    for path in paths:
        count += 1
        if path[0] != 0:
            continue
        cur_weight = weight_of_path(G, path)
        if min_weight > cur_weight:
            min_weight = cur_weight
            min_path = path
    return min_weight, min_path

In [3]:
# Не нужна
def print_concorde_input(G):
    tsp_template = """
        TYPE : TSP
        DIMENSION : %i
        EDGE_WEIGHT_TYPE : EXPLICIT
        EDGE_WEIGHT_FORMAT : LOWER_DIAG_ROW 
        EDGE_WEIGHT_SECTION
        %s
        EOF
        """
    edges = [(u, v, G[u][v]['weight']) for (u, v) in G.edges]
    print(edges)
    return tsp_template%(G.number_of_vertices(), edges)

In [4]:
# TODO: почему-то нет пути()
def generate_12_euclidean_TSP(n, side, n_iter, p1, p2):
    # TODO: для многих итераций точки могут совпадать.
    G = nx.Graph()
    for i in range(0, n):
        G.add_node(i)
        G.nodes[i]['pos'] = [random.uniform(0, side), random.uniform(0, side)]
    
    def intersect_circles(a, b, r_1, c, d, r_2):
        #Расстояние между центрами
        D = np.sqrt((c-a)**2+(d-b)**2)
        if not(r_1+r_2>D and D>np.abs(r_1-r_2)):
            return None

        delta = 1/4*np.sqrt((D+r_1+r_2)*(D+r_1-r_2)*(D-r_1+r_2)*(-D+r_1+r_2))

        x = (a+c)/2+(c-a)*(r_1**2-r_2**2)/(2*D**2)
        x_add = 2*(b-d)/D**2*delta
        y = (b+d)/2+(d-b)*(r_1**2-r_2**2)/(2*D**2)
        y_add = 2*(a-c)/D**2*delta

        return ((x+x_add, y-y_add),(x-x_add, y+y_add))

    def draw_edge(u, v, r):
        if random.uniform(0, 1)<p2:
                    G.add_edge(u, v, weight=r)
    
    def check_intersection(a, b, c, d, r):
        points = intersect_circles(a, b, r, c, d, r)
        if points is not None:
            if random.uniform(0, 1)<p1:
                new_node = G.number_of_nodes()
                G.add_node(new_node, pos=points[0])
                
                draw_edge(i, new_node, r)
                draw_edge(j, new_node, r)
            if random.uniform(0, 1)<p1:
                new_node = G.number_of_nodes()
                G.add_node(new_node, pos=points[1])
                
                draw_edge(i, new_node, r)
                draw_edge(j, new_node, r)
            #print(i, j, points)
    
    for cur_iter in range(0, n_iter):
        for i in range(0, n):
            for j in range(0, i):
                a, b = G.nodes[i]['pos']
                c, d = G.nodes[j]['pos']
                check_intersection(a, b, c, d, 1)
                check_intersection(a, b, c, d, 2)
    
    # Удалям все висячие вершины.
    deletes = True
    while deletes:
        deletes = False
        to_delete = []
        for n in G.nodes():
            if len(G.edges(n))<=1:
                to_delete.append(n)
                deletes = True
        G.remove_nodes_from(to_delete)

    return G

G = None
while True:
    G = generate_12_euclidean_TSP(20, 10, 1, 0.3, 1)
    #draw_weighted_graph(G, with_labels=True)
    #print(G.number_of_nodes(), G.number_of_edges())
    try:
        n, path = solve_with_concorde(G)
        #print(weight_of_path(G, path))
        if weight_of_path(G, path) < math.inf:
            break
    except Exception:
        pass
    #print(weight_of_path(G, path))
    #print([G.nodes[i] for i in range(G.number_of_nodes())])
plt.figure(figsize=(15, 10))
draw_weighted_graph(G, with_labels=True)

NameError: name 'nx' is not defined

In [ ]:
# Вообще бред какой-то
def generate_12_euclidean_TSP(n_init, side, n_iter, p1, p2):
        
    def get_node_at_pos(G, pos):
        for a in G.nodes:
            if G.nodes[a]['pos'][0] == pos[0] and G.nodes[a]['pos'][1] == pos[1]:
                return a
        return None

    def insert_to_pos(G, node, pos):
        a = get_node_at_pos(G, pos)
        if a is None and random.uniform(0, 1) < p1:
            added = G.number_of_nodes()
            G.add_node(added, pos=pos)
            G.add_edge(node, added, weight=1)
        elif a is not None and random.uniform(0, 1) < p2:
            pass
#         for (u, v) in G.edges:
#                 if 
    
    def draw_edge(G, n, r, p, pos):
        if random.uniform(0, 1) < p:
            a = get_node_at_pos(G, pos)
            if a is not None and not G.has_edge(n, a):
                G.add_edge(n, a, weight=r)
            
    def draw_edges(G, n, r, p):
        d = G.nodes[n]
        draw_edge(G, n, r, p, (d['pos'][0]-r, d['pos'][1]))
        draw_edge(G, n, r, p, (d['pos'][0]+r, d['pos'][1]))
        draw_edge(G, n, r, p, (d['pos'][0], d['pos'][1]-r))
        draw_edge(G, n, r, p, (d['pos'][0], d['pos'][1]+r))
    
    G = nx.Graph()
    for i in range(0, n_init):
        pos = [random.randint(0, side), random.randint(0, side)]
        if get_node_at_pos(G, pos) is not None:
            continue
        added = G.number_of_nodes()
        G.add_node(added)
        G.nodes[added]['pos'] = pos
    
    for i in range(0, n_iter):
        n = random.randint(0, G.number_of_nodes()-1)
        draw_edges(G, n, 1, p1)
        draw_edges(G, n, 2, p2)
#         d = G.nodes[node]

#         insert_to_pos(G, node, (d['pos'][0]-1, d['pos'][1]))
#         insert_to_pos(G, node, (d['pos'][0]+1, d['pos'][1]))
#         insert_to_pos(G, node, (d['pos'][0], d['pos'][1]-1))
#         insert_to_pos(G, node, (d['pos'][0], d['pos'][1]+1))
    
    print(G.number_of_nodes(), G.number_of_edges())
    deletes = True
    while deletes:
        deletes = False
        to_delete = []
        for n in G.nodes():
            if len(G.edges(n))<=1:
                to_delete.append(n)
                deletes = True
        for n in to_delete:
            G.remove_edges_from(list(G.edges(n)))
        G.remove_nodes_from(to_delete)
        
    print(G.number_of_nodes(), G.number_of_edges())
    return G

G = None
while True:
    G = generate_12_euclidean_TSP(100, 10, 150, 0.8, 0.5)
    #draw_weighted_graph(G, with_labels=True)
#     G = nx.generators.random_geometric_graph(n=30, radius=0.3)
#     for (u, v) in G.edges():
#         G[u][v]['weight'] = random.randint(1, 2)
    #print(G.number_of_nodes(), G.number_of_edges())
    try:
        n, path = solve_with_concorde(G)
        #print(weight_of_path(G, path))
        if weight_of_path(G, path) < math.inf:
            break
    except Exception:
        pass
    #print(weight_of_path(G, path))
    #print([G.nodes[i] for i in range(G.number_of_nodes())])
plt.figure(figsize=(15, 10))
draw_weighted_graph(G, with_labels=True)

In [ ]:
def get_node_at_pos(G, pos):
    for a in G.nodes:
        if G.nodes[a]['pos'][0] == pos[0] and G.nodes[a]['pos'][1] == pos[1]:
            return a
    return None

def generate_walk(n, p1, p2):
    G = nx.Graph()
    pos = (0, 0)
    G.add_node(0, pos=pos)
    
    for i in range(1, n):
        r = 1 if random.uniform(0, 1) < p2 else 2
        pos_ = None
        while True:
            pr = random.uniform(0, 1)
            if pr < 0.25:
                pos_ = (pos[0]-r, pos[1])
            elif pr<0.5:
                pos_ = (pos[0]+r, pos[1])
            elif pr<0.75:
                pos_ = (pos[0], pos[1]-r)
            else:
                pos_ = (pos[0], pos[1]+r)
            if get_node_at_pos(G, pos_) is None:
                print(pos_)
                break
        G.add_node(i, pos=pos_)
        G.add_edge(i, i-1, weight=r)
        pos = pos_
    
    draw_weighted_graph(G, with_labels=True)
    return G

generate_walk(20, 0.5, 0.5)

In [ ]:
def powerset(iterable):
    xs = list(iterable)
    # note we return an iterator rather than a list
    return itertools.chain.from_iterable(itertools.combinations(xs,n) for n in range(len(xs)+1))

def manhattan_distance(G, i, j):
    posi = G.nodes[i]['pos']
    posj = G.nodes[j]['pos']
    return math.fabs(posi[0]-posj[0])+math.fabs(posi[1]-posj[1])

def to_12_tsp_manhattan(G):
    G_ = G.copy()
    for i in G_.nodes():
        for j in G_.nodes():
            if i != j and manhattan_distance(G, i, j) == 2:
                    if not G_.has_edge(i, j):
                        G_.add_edge(i, j, weight=2)
    return G_


In [ ]:
def generate_p_shaped(n, p1, p2):
    G = nx.Graph()
    
    for i in range(-1, n):
        for j in range(0, 4):
            G.add_node((j, i), pos=(j, i))
    
    for i in range(0, n-1):
        for j in range(0, 4):
            G.add_edge((j, i),(j, i+1),weight=1)
    
    G.add_edge((0, n-1), (1, n-1), weight=1)
    G.add_edge((2, n-1), (3, n-1), weight=1)
    G.add_edge((1, 0), (2, 0), weight=1)
    G.add_edge((0, 0), (0, -1), weight=1)
    G.add_edge((3, 0), (3, -1), weight=1)
    G.add_edge((0, -1), (1, -1), weight=1)
    for i in range(0, 3):
        G.add_edge((i, -1), (i+1, -1), weight=1)
    for i in range(1, n):
        for j in range(0, 3):
            if random.uniform(0, 1) < p1:
                G.add_edge((j, i), (j+1, i), weight=1)
    #for i in range(1, n):
    #    for j in range(0, 2):
    #        if random.uniform(0, 1) < p2:
    #            G.add_edge((j, i), (j+2, i), weight=2)
    return G


G = generate_p_shaped(3, 0.5, 0.5)
print(G.number_of_nodes(), G.number_of_edges())
process_tsp_manual(G, draw=False)

In [ ]:
def gen_grid(n, m, p):
    G = nx.Graph()
    for i in range(0, n):
        for j in range(0, m):
            G.add_node((i, j), pos=(i, j))
    def add_edges(n, m, r):
        n -= r
        m -= r
        for i in range(0, n):
            for j in range(0, m):
                if random.uniform(0, 1) < p:
                    G.add_edge((i, j), (i+r,j), weight=r)
                if random.uniform(0, 1) < p:
                    G.add_edge((i, j), (i,j+r), weight=r)
        for i in range(0, n):
            if random.uniform(0, 1) < p:
                G.add_edge((i, m), (i+r, m), weight=r)
        for j in range(0, m):
            if random.uniform(0, 1) < p:
                G.add_edge((n, j), (n, j+r), weight=r)
    add_edges(n, m, 1)
    add_edges(n, m, 2)
    #draw_weighted_graph(G, with_labels=True)
    G = set_edges_weight(G)
    return G
process_tsp_manual(gen_grid(4,4, 0.5))

In [ ]:
for j in range(0,40):
    K = min(j, 15)
    itern = int(sum([comb(j, i) for i in range(0, K+1)])-j-1)
    print(r"%s & %s & %s\\ \hline"%(j, itern, itern//1500//60))